## pandas

pandas是基于Numpy构建的，让以Numpy为中心的应用变得更加简单。

pandas引入约定：

In [1]:
from pandas import Series, DataFrame

In [2]:
import pandas as pd

### Series

Series是一种类似于一维数组的对象，它由一组数据以及索引组成，可以通过Series的values和index属性获取其数组表示形式和索引对象：

In [3]:
import numpy as np
obj = Series(np.arange(1,10,2))
obj

0    1
1    3
2    5
3    7
4    9
dtype: int32

In [4]:
obj.values

array([1, 3, 5, 7, 9])

In [5]:
obj.index

RangeIndex(start=0, stop=5, step=1)

Numpy数组运算（如根据布尔型数组进行过滤，标量乘法，应用数学函数等）都会保留索引和值之间的链接：

In [7]:
obj[obj%3==0]

1    3
4    9
dtype: int32

pandas的isnull和notnull函数可用于检测缺失数据：

In [8]:
pd.isnull(obj)

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [9]:
pd.notnull(obj)

0    True
1    True
2    True
3    True
4    True
dtype: bool

*pd.isnull(obj)等价于obj.isnull()*

Series对象本身及其索引都有一个name属性：

In [16]:
obj.name = 'obj'
obj.index.name = 'index'
obj

index
0    1
1    3
2    5
3    7
4    9
Name: obj, dtype: int32

### DataFrame

DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值，字符，布尔值等）。DataFrame既有行索引也有列索引，可以看作是由Series组成的字典（共用同一个索引）。

构建DataFrame的方法很多，最常用的是直接传入一个由等长列表或Numpy数组组成的字典：

In [11]:
data={
'year':[23,32,43,22],
'pop':np.arange(4)}
data

{'pop': array([0, 1, 2, 3]), 'year': [23, 32, 43, 22]}

In [12]:
frame=DataFrame(data)
frame

,pop,year
0,0,23
1,1,32
2,2,43
3,3,22


*如果指定离列序列，则DataFrame的列就会按照指定顺序进行排列，如：DataFrame（data,columns=['year','pop']*

通过类似字典标记的方式或属性的方式，可以将DataFrame的列获取为一个Series：

In [13]:
frame.year

0    23
1    32
2    43
3    22
Name: year, dtype: int64

为不存在的列赋值会创建出一个新列，如果赋值的是一个Series，就会精确匹配DataFrame的索引，空位将被填上缺失值：

In [14]:
val = Series([2,2,2],index=[0,2,4])
val

0    2
2    2
4    2
dtype: int64

In [15]:
frame['debt']

,pop,year,debt
0,0,23,2.0
1,1,32,NaN
2,2,43,2.0
3,3,22,NaN


关键字del用于删除列：

In [17]:
del frame['pop']
frame

,year,debt
0,23,2.0
1,32,NaN
2,43,2.0
3,22,NaN


嵌套字典（字典中的字典）：另一种常见的数据形式，将它传给DataFrame，它就会被解释为：外层字典的键作为列，内层键则作为行索引

In [18]:
pop = {'NNN':{2001:2.4,2002:2.9},
      'OOO':{2000:1.5,2001:1.8,2002:3.6}}
DataFrame(pop)

,NNN,OOO
2000,NaN,1.5
2001,2.4,1.8
2002,2.9,3.6


*当然也可以对结果进行转置.T，以及显式指定索引*

### 索引对象

pandas的索引对象负责管理轴标签和其它元数据(比如轴名称等）。构建Series和DataFrame时，所用到的任何数组或其它序列标签都会被转换成一个Index:

In [19]:
obj = Series(range(3),index=['a','b','c'])
obj.index

Index(['a', 'b', 'c'], dtype='object')

Index的对象是不可修改的，这样才能使Index对象在多个数据结构之间安全共享：

In [20]:
obj.index[1]='d'

TypeError: Index does not support mutable operations

### 基本功能
*一些操作Series和DataFram中的数据的基本手段*

重新索引：pandas对象的一个重要方法是reindex,其作用是创建一个适应新索引的新对象

In [21]:
obj = Series([1.1,2.2,3.3,4.4],index=['d','b','a','c'])

obj

d    1.1
b    2.2
a    3.3
c    4.4
dtype: float64

In [25]:
obj.reindex(['a','b','c','d','e','f'])

a    3.3
b    2.2
c    4.4
d    1.1
e    NaN
f    NaN
dtype: float64

In [27]:
#reindex(index,method,column)
#缺失值填充为0
obj.reindex(['a','b','c','d','e','f'],fill_value=0)

a    3.3
b    2.2
c    4.4
d    1.1
e    0.0
f    0.0
dtype: float64

In [30]:
obj2 = Series(['blue','purple','yellow'],index=[0,2,4])
#ffill实现前向值填充
obj2.reindex(range(6),method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

*ffill&pad 前向 bfill&backfill 后向
插值只能按轴0应用*

标签索引：利用ix可以使重新索引任务变得更简洁

In [33]:
frame.ix[range(4),['debt','year']]

,debt,year
0,2.0,23
1,NaN,32
2,2.0,43
3,NaN,22


### 丢弃指定轴上的项：